In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin.gz
/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin
/kaggle/input/disasters-on-social-media/socialmedia-disaster-tweets-DFE.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/glove-embeddings/glove.6B.200d.txt
/kaggle/input/glove-embeddings/glove.6B.50d.txt
/kaggle/input/glove-embeddings/glove.6B.300d.txt
/kaggle/input/glove-embeddings/glove.6B.100d.txt


In [2]:
import pandas as pd

data=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
data2 = pd.read_csv('/kaggle/input/disasters-on-social-media/socialmedia-disaster-tweets-DFE.csv')[["keyword","location","text","choose_one"]]
data2.rename(columns={"choose_one":"target"},inplace=True)
data2["target"] = (data2["target"] == "Relevant").astype("int")

In [3]:
print(data.columns)
print(data2.columns)

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
Index(['keyword', 'location', 'text', 'target'], dtype='object')


# Concatenating two datsets 

In [4]:
import pandas as pd

# Assuming data and data2 are already loaded as pandas DataFrames

# Selecting only 'text' and 'target' columns from both dataframes
data = pd.concat([data[['text', 'target']], data2[['text', 'target']]], ignore_index=True)

# Display the first few rows
data


,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
18484,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
18485,Police investigating after an e-bike collided ...,1
18486,The Latest: More Homes Razed by Northern Calif...,1
18487,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [5]:
print(data.isnull().sum())  # Check for missing values

text      0
target    0
dtype: int64


In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from sklearn.model_selection import train_test_split
import re
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Initialize stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))


def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)  # Removes anything starting with 'http' or 'www'

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Join the tokens back into a single string
    return ' '.join(tokens)

# Apply preprocessing to the 'text' column
data['processed_text'] = data['text'].apply(preprocess_text)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['processed_text'], data['target'], test_size=0.2, random_state=42)

# Display the processed data
X_train.head()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


5937                         i have never screamed so loud
6991     reasons brain twister oneself should discount ...
1433     another sad ocean casualtygray whale populatio...
177      twelve feared killed in pakistani air ambulanc...
17937    on 70th anniversary of hiroshima bomb it is im...
Name: processed_text, dtype: object

In [7]:
import pandas as pd  

# Assuming y contains class labels
print(y_train.value_counts())  

# To get percentages
print(y_train.value_counts(normalize=True) * 100)


target
0    8421
1    6370
Name: count, dtype: int64
target
0    56.93327
1    43.06673
Name: proportion, dtype: float64


In [8]:
X_train[:1]

5937    i have never screamed so loud
Name: processed_text, dtype: object

In [9]:
len(X_train[:1])

1

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train the model
model_tfidf = LogisticRegression()
model_tfidf.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = model_tfidf.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8385613845321795
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.92      0.87      2124
           1       0.87      0.73      0.79      1574

    accuracy                           0.84      3698
   macro avg       0.85      0.82      0.83      3698
weighted avg       0.84      0.84      0.84      3698



In [13]:
import pandas as pd

# Find misclassified indices
misclassified_indices = [i for i, (true, pred) in enumerate(zip(y_test, y_pred)) if true != pred]

# Extract misclassified examples
misclassified_data = pd.DataFrame({
    "Text": X_test.iloc[misclassified_indices],
    "True Label": y_test.iloc[misclassified_indices],
    "Predicted Label": y_pred[misclassified_indices]
})

# Display misclassified examples
print("\nMisclassified Examples:")
print(misclassified_data.head(10))  # Show first 10 misclassified examples

# Analyze feature importance
feature_names = vectorizer.get_feature_names_out()
coef = model_tfidf.coef_[0]  # Coefficients of logistic regression

# Find top positive and negative words
top_positive_words = [feature_names[i] for i in coef.argsort()[-10:]]  # 10 words most strongly predicting positive class
top_negative_words = [feature_names[i] for i in coef.argsort()[:10]]   # 10 words most strongly predicting negative class

print("\nTop Words Indicating Positive Class:", top_positive_words)
print("Top Words Indicating Negative Class:", top_negative_words)



Misclassified Examples:
                                                    Text  True Label  \
733          katramsland yes im a bleeding heart liberal           1   
11586  is this country latin americas next argentina ...           0   
3479   exploring new worlds three moments of an explo...           1   
273    ohh no fukurodani didnt survive the apocalypse...           1   
15275  elielcruz just watching the streams was bad i ...           1   
17576  freefromwolves godslove amp thanku brother dan...           0   
8803   ashayo msmiggi hi ashayo i believe there will ...           1   
14905  over half of poll respondents worry nuclear di...           0   
13064  please pray for employees residents and first ...           0   
3532                a true tbt eyewitness news wbre wyou           0   

       Predicted Label  
733                  0  
11586                1  
3479                 0  
273                  0  
15275                0  
17576                1  
8803   

# Reasons for misclassification
1. Loss of Context & Word Order
2. Failure to Capture Semantic Meaning
3. Presence of Ambiguous or Common Words

In [14]:
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK data (if not already downloaded)
nltk.download('punkt')

# Tokenize the text
X_train_tokens = [word_tokenize(text) for text in X_train]
X_test_tokens = [word_tokenize(text) for text in X_test]

from gensim.models import KeyedVectors

# Load pre-trained Word2Vec embeddings (e.g., Google News)
word2vec_model = KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)

# Save the model as KeyedVectors
word2vec_model.save("word2vec.model")

# Load the model as KeyedVectors
word2vec_model = KeyedVectors.load("word2vec.model", mmap='r')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
import numpy as np

def sentence_vector(tokens, model, vector_size=300):  # Set vector_size to match the embedding dimension 
    vec = np.zeros(vector_size)  # Initialize a vector of zeros
    count = 0  # Count of words with valid vectors
    for word in tokens:
        if word in model:  # Check if the word is in the KeyedVectors vocabulary
            vec += model[word]  # Add the word vector
            count += 1
    if count != 0:
        vec /= count  # Average the vectors
    return vec

# Convert tokenized sentences to sentence vectors
X_train_vectors = np.array([sentence_vector(tokens, word2vec_model) for tokens in X_train_tokens])
X_test_vectors = np.array([sentence_vector(tokens, word2vec_model) for tokens in X_test_tokens])

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train the model
model_w2v = LogisticRegression()
model_w2v.fit(X_train_vectors, y_train)

# Evaluate the model
y_pred_w2v = model_w2v.predict(X_test_vectors)
print("Accuracy:", accuracy_score(y_test, y_pred_w2v))
print("Classification Report:\n", classification_report(y_test, y_pred_w2v))

Accuracy: 0.8109789075175771
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.88      0.84      2124
           1       0.82      0.71      0.76      1574

    accuracy                           0.81      3698
   macro avg       0.81      0.80      0.80      3698
weighted avg       0.81      0.81      0.81      3698



In [21]:
# Reset index to ensure consistent integer indexing
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Ensure predictions are also converted to arrays if needed
y_pred = np.array(y_pred)
y_pred_w2v = np.array(y_pred_w2v)

# Debugging: Check what indices exist in X_test
valid_indices = set(X_test.index)
i=0
# Print the misclassified samples with their true labels and predictions
for idx in wrong_tfidf_correct_w2v:
    if(i>2): break
    if idx in valid_indices:
        print(f"Sample Sentence: {X_test.iloc[idx]}")
        print(f"True Label: {y_test.iloc[idx]}, TF-IDF Prediction: {y_pred[idx]}, W2V Prediction: {y_pred_w2v[idx]}\n")
    else:
        print(f"Warning: Index {idx} not found in X_test. Skipping...")
    i+=1

Sample Sentence: a true tbt eyewitness news wbre wyou
True Label: 0, TF-IDF Prediction: 1, W2V Prediction: 0

Sample Sentence: enugu government to demolish illegal structures at international conference centre
True Label: 1, TF-IDF Prediction: 0, W2V Prediction: 1

Sample Sentence: 19000 homes destroyed in gaza zero rebuilt join me in busting the blockade sign amp rt gaza opengaza
True Label: 1, TF-IDF Prediction: 0, W2V Prediction: 1



In [22]:
import pandas as pd
import numpy as np

# Create a DataFrame for easier analysis
misclassified_df = pd.DataFrame({
    'Text': X_test, 
    'True Label': y_test, 
    'Predicted Label': y_pred
})

# Filter only misclassified examples
misclassified_df = misclassified_df[misclassified_df['True Label'] != misclassified_df['Predicted Label']]

# Display misclassified examples
print("Misclassified Examples:")
print(misclassified_df)


Misclassified Examples:
                                                   Text  True Label  \
3           katramsland yes im a bleeding heart liberal           1   
13    is this country latin americas next argentina ...           0   
19    exploring new worlds three moments of an explo...           1   
23    ohh no fukurodani didnt survive the apocalypse...           1   
27    elielcruz just watching the streams was bad i ...           1   
...                                                 ...         ...   
3677                    soonergrunt better than tornado           0   
3678  esteemed journalist recalls tragic effects of ...           1   
3681  bigamist and his ‰û÷first‰ûª wife are charged ...           1   
3688  also confirmed by senate marie therese domingu...           0   
3696  like bro they just committed a mass murder you...           1   

      Predicted Label  
3                   0  
13                  1  
19                  0  
23                  0  
27 

# Reasons for misclassification
1. Failure to Capture Sentence Context 
2. Vocabulary Coverage & OOV (Out-of-Vocabulary) Words
3. Does Not Account for Sentence Structure

In [23]:
!!pip install wandb


['Requirement already satisfied: wandb in /usr/local/lib/python3.10/dist-packages (0.19.1)',
 'Requirement already satisfied: click!=8.0.0,>=7.1 in /usr/local/lib/python3.10/dist-packages (from wandb) (8.1.7)',
 'Requirement already satisfied: docker-pycreds>=0.4.0 in /usr/local/lib/python3.10/dist-packages (from wandb) (0.4.0)',
 'Requirement already satisfied: gitpython!=3.1.29,>=1.0.0 in /usr/local/lib/python3.10/dist-packages (from wandb) (3.1.43)',
 'Requirement already satisfied: platformdirs in /usr/local/lib/python3.10/dist-packages (from wandb) (4.3.6)',
 'Requirement already satisfied: protobuf!=4.21.0,!=5.28.0,<6,>=3.19.0 in /usr/local/lib/python3.10/dist-packages (from wandb) (3.20.3)',
 'Requirement already satisfied: psutil>=5.0.0 in /usr/local/lib/python3.10/dist-packages (from wandb) (5.9.5)',
 'Requirement already satisfied: pydantic<3,>=2.6 in /usr/local/lib/python3.10/dist-packages (from wandb) (2.11.0a2)',
 'Requirement already satisfied: pyyaml in /usr/local/lib/py

In [24]:
import wandb
wandb.login(key="4bf281fc3d5b1a088a9793a64f35253a6c2d7bc6")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: indonesia240873 (indonesia240873-shiv-nadar-university-chennai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [25]:
!pip install transformers datasets scikit-learn


In [26]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)  # Move model to GPU if available

# Function to extract BERT embeddings using batch processing
def get_bert_embeddings(text_list, batch_size=16):
    embeddings = []
    
    # Ensure all inputs are strings
    text_list = [str(text) for text in text_list]

    for i in tqdm(range(0, len(text_list), batch_size), desc="Extracting BERT embeddings"):
        batch_texts = text_list[i:i + batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = bert_model(**inputs)
        
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Get CLS token
        embeddings.append(batch_embeddings)
    
    return np.vstack(embeddings)  # Stack all embeddings into a single array

# Convert data to lists of strings
X_train = [str(x) for x in X_train]  # Convert each element to string
X_test = [str(x) for x in X_test]    # Convert each element to string

# Extract BERT embeddings
X_train_embedded = get_bert_embeddings(X_train, batch_size=16)
X_test_embedded = get_bert_embeddings(X_test, batch_size=16)

# Train Logistic Regression Model
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train_embedded, y_train)

# Make pre_dictions
y_pred_bemb = logreg.predict(X_test_embedded)

# Convert y_test to NumPy array to avoid KeyError
y_test = np.array(y_test)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred_bemb)
print(f"Test Accuracy: {accuracy:.4f}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Extracting BERT embeddings: 100%|██████████| 232/232 [00:05<00:00, 38.90it/s]


Test Accuracy: 0.8067


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
# Make pre_dictions
y_pred_bemb = logreg.predict(X_test_embedded)

# Convert y_test to NumPy array to avoid KeyError
y_test = np.array(y_test)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred_bemb)
print(f"Test Accuracy: {accuracy:.4f}")

# Convert predictions and y_test to NumPy arrays if not already
y_pred_w2v = np.array(y_pred_w2v)
y_pred_bemb = np.array(y_pred_bemb)
y_test = np.array(y_test)

# Find indices where Word2Vec was incorrect, but Logistic Regression was correct
wrong_w2v_correct_bemb = [
    idx for idx in range(len(y_test)) if (y_pred_w2v[idx] != y_test[idx]) and (y_pred_bemb[idx] == y_test[idx])
]

# Print the misclassified samples
print("Samples misclassified by Word2Vec but classified correctly by Logistic Regression:\n")
for idx in wrong_w2v_correct_bemb[:5]:  # Limit output to 5 samples for readability
    print(f"Sample Sentence: {X_test[idx]}")  # Use direct indexing
    print(f"True Label: {y_test[idx]}, W2V Prediction: {y_pred_w2v[idx]}, BERT/Embeddings Prediction: {y_pred_bemb[idx]}\n")



Test Accuracy: 0.8067
Samples misclassified by Word2Vec but classified correctly by Logistic Regression:

Sample Sentence: ‰ûïrichmond coaches were devastated to hear of the death of their second driver mr chance who was sitting‰û jam
True Label: 1, W2V Prediction: 0, BERT/Embeddings Prediction: 1

Sample Sentence: gracemccreave jamisonevann jaceswain laneyslife jacksonlaurie1 lava boy was taken
True Label: 0, W2V Prediction: 1, BERT/Embeddings Prediction: 0

Sample Sentence: exploring new worlds three moments of an explosion by china miì©ville
True Label: 1, W2V Prediction: 0, BERT/Embeddings Prediction: 1

Sample Sentence: freefromwolves godslove amp thanku brother danny for rt of new video the coming apocalyptic us earthquake amp tsunami
True Label: 0, W2V Prediction: 1, BERT/Embeddings Prediction: 0

Sample Sentence: arianagrande screamqueens katherines death
True Label: 0, W2V Prediction: 1, BERT/Embeddings Prediction: 0



In [39]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# Define accuracy metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ensure X_train and X_test are lists of strings
X_train = [str(text) for text in X_train]
X_test = [str(text) for text in X_test]

# Tokenize the data
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)

# Ensure labels are lists
y_train = list(y_train)
y_test = list(y_test)

# Custom PyTorch dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Convert labels to LongTensor
        return item
    
    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = Dataset(train_encodings, y_train)
test_dataset = Dataset(test_encodings, y_test)


model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",  # Use accuracy to pick best model
    greater_is_better=True,  # Higher accuracy is better
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Add accuracy metric
)

# Train and evaluate the model
trainer.train()
trainer.evaluate()

# Save the trained model and tokenizer
model.save_pretrained("./bert_text_classifier")
tokenizer.save_pretrained("./bert_text_classifier")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.416000,0.329045,0.875608
2,0.186500,0.309634,0.900757
3,0.114300,0.336176,0.911033
4,0.096500,0.371244,0.917253
5,0.090600,0.421626,0.920227
6,0.096200,0.436285,0.919957
7,0.024200,0.534951,0.920498
8,0.013400,0.525446,0.927528
9,0.000300,0.555860,0.925906
10,0.019700,0.586688,0.926447


('./bert_text_classifier/tokenizer_config.json',
 './bert_text_classifier/special_tokens_map.json',
 './bert_text_classifier/vocab.txt',
 './bert_text_classifier/added_tokens.json')

In [40]:
# Get model predictions
predictions = trainer.predict(test_dataset).predictions
predicted_labels = np.argmax(predictions, axis=-1)

# Identify misclassified samples
misclassified_indices = np.where(predicted_labels != np.array(y_test))[0]

print("\nMisclassified Samples:")
for idx in misclassified_indices:
    print("-" * 50)
    print(f"Sample {idx+1}:")
    print(f"  Text: {X_test[idx]}")
    print(f"  True Label: {y_test[idx]}")
    print(f"  Predicted Label: {predicted_labels[idx]}")
print("-" * 50)



Misclassified Samples:
--------------------------------------------------
Sample 4:
  Text: katramsland yes im a bleeding heart liberal
  True Label: 1
  Predicted Label: 0
--------------------------------------------------
Sample 24:
  Text: ohh no fukurodani didnt survive the apocalypse bokuto feels horrible my poor boy my ppor child
  True Label: 1
  Predicted Label: 0
--------------------------------------------------
Sample 40:
  Text: people tilly the confused cat overcomes horrible attack to win hearts httptcoqtrsyxfzo3
  True Label: 1
  Predicted Label: 0
--------------------------------------------------
Sample 50:
  Text: enugu government to demolish illegal structures at international conference centre httptcoouylwuixcs
  True Label: 1
  Predicted Label: 0
--------------------------------------------------
Sample 60:
  Text: creationmin rwrabbit gooneratheist atheistic1 lolatjesus yet still why did so many fish die in the worldwide flood i wonder
  True Label: 0
  Predicted